In [25]:
import torch
print(torch.cuda.is_available())

True


In [26]:
import scipy.io
mat = scipy.io.loadmat('C:/Users/start/Documents/dataset/SSVEPdataset.mat')

In [35]:
data = torch.tensor(mat['lowFrequencyData'], dtype=torch.float32)
labels = torch.tensor(mat['lowLabels'], dtype=torch.long)
labels = labels.squeeze()

In [36]:
from torch.utils.data import TensorDataset, DataLoader

dataset = TensorDataset(data, labels)

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [37]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SSVEPConvModel(nn.Module):
    def __init__(self):
        super(SSVEPConvModel, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=39, out_channels=64, kernel_size=3)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3)
        self.pool = nn.MaxPool1d(2,2)
        calculated_output_size = 128 * 298
        self.fc1 = nn.Linear(calculated_output_size, 256)
        self.fc2 = nn.Linear(256,4)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)

        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)

        x = x.view(x.size(0), -1)
        
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.softmax(x, dim=1)
        
        return x

model = SSVEPConvModel()

In [38]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [39]:
# training function and test function
def train(model, train_loader, optimizer, criterion):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    return running_loss / len(train_loader.dataset)

def test(model, test_loader, criterion):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

In [40]:
# train and test the model
num_epochs = 5
for epoch in range(num_epochs):
    train_loss = train(model, train_loader, optimizer, criterion)
    test_accuracy = test(model, test_loader, criterion)
    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

Epoch [1/5], Train Loss: 1.4895, Test Accuracy: 0.2490
Epoch [2/5], Train Loss: 1.4934, Test Accuracy: 0.2490
Epoch [3/5], Train Loss: 1.4934, Test Accuracy: 0.2490
Epoch [4/5], Train Loss: 1.4934, Test Accuracy: 0.2490
Epoch [5/5], Train Loss: 1.4934, Test Accuracy: 0.2490
